In [2]:
import findspark
findspark.init("D:\spark\spark-2.4.3-bin-hadoop2\spark-2.4.3-bin-hadoop2.7")
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf

sc = SparkContext("local", "Simple")

In [17]:
def contains(line):
    return "hello world" in line

# 读取windows文件(统计词出现的次数)
fileRDD = sc.textFile("file:///C:/Users/wd/Documents/GitHub/Datawhale_DA/testSpark.txt")
filterRDD = fileRDD.filter(contains)
filterRDD.cache()
print (filterRDD.count())

#
sc.stop()
#filterRDD.stop()

3


In [23]:
sc.stop()
sc = SparkContext("local", "Simple")
#fileRDD = sc.textFile("file:///C:/Users/wd/Documents/GitHub/Datawhale_DA/testSpark.txt")
#fileRDD.head()
#fileRDD.foreach(print)
fileDF = sc.read.text("file:///C:/Users/wd/Documents/GitHub/Datawhale_DA/testSpark.txt")
fileDF.head()
sc.stop()

AttributeError: 'SparkContext' object has no attribute 'read'

In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
#启动spark
spark = SparkSession.builder.master("local").appName("test").getOrCreate()
sc = spark.sparkContext

In [27]:
textFromFile = spark.read.text("C:/Users/wd/Documents/GitHub/Datawhale_DA/testSpark.txt")
textFromFile.head()

Row(value='hello world!!!')

In [29]:
#--- 显示对应的值 ---
textFromFile.show()

+--------------------+
|               value|
+--------------------+
|      hello world!!!|
|print hello world...|
|     hello world!!!!|
+--------------------+



In [31]:
# 再次读取csv文件
csvDF = spark.read.csv('test_data.csv', sep=',', header=True, schema=None)
csvDF.show()

+----+----------+------------------+---------------------+-------------------------------------+---------------------------------+--------------------+-------------------+----------+--------------------------+-------------------+-----------------+---------------+--------------------+------------+-------------------------+----------------------+-----------------------+--------------------+-----------------+--------------------+-------------------------------+------+----------------------------+-----------------------+---------------------------------+-----------------------+---------------------------+---------------------------+-----------------------------+-----------------------------+--------------------------------+--------------------------------------+--------------------------------------+---------------------------------------+------------------------------+------------------------------------+-------------------------------+-----------------------------------+-----------------

In [39]:
# 转换成df 然后再进行计算
csvDF.select("low_volume_percent").show(5)

+------------------+
|low_volume_percent|
+------------------+
|              0.03|
|              0.02|
|               0.0|
|              0.01|
|              0.02|
+------------------+
only showing top 5 rows



In [1]:
# 计算排名
def mapToPairs(line):
    res = line.split(',')
    return (res[0], float(res[1]))

def add(a, b):
    return a + b

# 对每个partition取前top k
def select_top_k(it):
    k = broadcast_k.value
    sorted_res = sorted(it, key=lambda x:x[1], reverse=True)
    top_k_res = [0] * k
    i = 0
    for t in sorted_res:
        top_k_res[i] = t
        i += 1
        if i >= k:
            break
    return top_k_res

# 先编写一列写入到数据中 然后再做rdd计算
data_path = 'data/arrays.txt'
out_path = 'sortResult.txt'
k_value = 10

# 设置master url 初始化sc
sc = SparkContext('local', 'TopK')
broadcast_k = sc.broadcast(k_value)
rdd = sc.textFile(data_path).map(mapToPairs).reduceByKey(add).mapPartitions(select_top_k)
top_k_res = rdd.takeOrdered(k_value, key=lambda x:-x[1])
#输出结果
with open(out_path, 'w') as out:
    for i in top_k_res:
        out.write(i[0] + '\n')

NameError: name 'SparkContext' is not defined

In [19]:
# Spark RDD计算排名PAT
from pyspark import SparkContext
#启动spark
spark = SparkSession.builder.master("local").appName("test").getOrCreate()
sc = spark.sparkContext


lines = sc.textFile("PatJudge.txt")

r = lines.map(lambda line:line.split(" ")).map(lambda line:((line[0],line[1]),line[2]))
pairRDD = r.reduceByKey(lambda a,b:a if a>b  else b)
#id和分数组一个RDD
pairRDD = pairRDD.map(lambda line:(line[0][0],line[1]))
#计算每个id的总分
SumVal = pairRDD.reduceByKey(lambda a,b:int(a)+int(b) if b!="-1" else int(a)+0)
SumVal.mapValues(lambda a,b:a>b)
#对相同key的id进行分组
EachVal = pairRDD.groupByKey()#.map(lambda line:(line[0],list(line[1]))).collect()
#把总分和每道题的得分合并，并且list化每一项的分数方便输出
zf = SumVal.join(EachVal).map(lambda line:(line[0],line[1][0],list(line[1][1]))).collect()
for i,j,val in zf:
    print(i,j,val)

7 20 ['20']
20 25 ['25']
00002 63 ['25', '20', '18']
00007 42 ['17', '25']
00005 42 ['20', '22', '-1']
00001 42 ['18', '18', '4', '2']
00004 40 ['25', '15', '0']
00006 -1 ['-1']
